## Clean the input shear catalogs

In [ ]:
import numpy as np
import tables_io

In [ ]:
def split_by_type_and_clean(basefile, tract, catType, types=['ns', '1m', '2m', '1p', '2p'], shear=0.01):
    p = tables_io.read(basefile)
    for type_ in types:
        mask = p['shear_type'] == type_
        sub = p[mask].copy(deep=True)
        idx_x = (20*sub['patch_x'].values + sub['cell_x'].values).astype(int)
        idx_y = (20*sub['patch_y'].values + sub['cell_y'].values).astype(int)
        cent_x = 150*idx_x -75
        cent_y = 150*idx_y -75
        xCellCoadd = sub['col'] - cent_x
        yCellCoadd = sub['row'] - cent_y
        sub["xCellCoadd"] = xCell_coadd
        sub["yCellCoadd"] = yCell_coadd
        sub["SNR"] = sub[f"{catType}_band_flux_r"] / sub[f"{catType}_band_flux_err_r"]
        sub["g_1"] = sub[f"{catType}_g_1"]
        sub["g_2"] = sub[f"{catType}_g_2"]
        sub["cellIdxX"] = idx_x
        sub["cellIdxY"] = idx_y
        sub["orig_id"] = sub.id
        sub["id"] = np.arange(len(sub))
        central_to_cell = np.bitwise_and(np.fabs(xCell_coadd) < 75, np.fabs(yCell_coadd) < 75)
        #central_to_cell = np.bitwise_and(np.fabs(xCell_coadd) < 80, np.fabs(yCell_coadd) < 80)
        central_to_patch = np.bitwise_and(np.fabs(sub['cell_x'].values - 10.5) < 10, np.fabs(sub['cell_y'].values - 10.5) < 10)
        right_tract = sub['tract'] == tract
        central = np.bitwise_and(central_to_cell, central_to_patch)
        selected = np.bitwise_and(right_tract, central)
        cleaned = sub[selected].copy(deep=True)
        cleaned['shear'] = np.repeat(shear, len(cleaned))
        cleaned.to_parquet(basefile.replace('.parq', f"_cleaned_{tract}_{type_}.pq"))

In [ ]:
shear_value_strs = ['0p0025', '0p005', '0p01', '0p02', '0p04']
shear_values = [0.0025, 0.005, 0.01, 0.02, 0.04]
cat_types = ['wmom', 'gauss', 'pgauss']
tracts = [10463, 10705]

In [ ]:
for tract_ in tracts:
    for shear_st_, shear_ in zip(shear_value_strs, shear_values):
        for cat_type_ in cat_types:
            split_by_type_and_clean(f"shear_tract/shear_{cat_type_}_{shear_st_}.parq", tract_, cat_type_, shear=shear_)